In [3]:

import pandas as pd

from transformers import AutoModelForCausalLM, AutoProcessor,AutoTokenizer
from utils import get_images



In [4]:
import torch
from transformers import BitsAndBytesConfig

config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)



In [5]:


model_id = "meta-llama/Llama-3.1-8B-Instruct" #

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    quantization_config=config
)
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)




Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = 128002



In [7]:
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  
    r=8, lora_alpha=32, 
    lora_dropout=0.1,
)
model = get_peft_model(model, peft_config)

In [8]:
import json

with open("messages_100.json", "r") as file:
    messages = json.load(file)
print(messages[0])
processor = AutoProcessor.from_pretrained(model_id)
input_text = [processor.apply_chat_template(message, tokenize=False) for message in messages]




[{'role': 'system', 'content': 'You are a helpful AI assistant in Material Science.'}, {'role': 'user', 'content': 'How to synthesize La4Ti9O24?'}, {'role': 'assistant', 'content': 'To synthesize the target material La4Ti9O24 through a precipitation pathway using the precursors La(NO3)3·6H2O and TiCl4, we can follow the detailed step-by-step synthesis procedure as outlined below:\n\n### Materials Needed:\n\n1. **Precursors:**\n   - Lanthanum Nitrate Hexahydrate, La(NO3)3·6H2O\n   - Titanium Tetrachloride, TiCl4\n\n2. **Solvent:**\n   - Distilled Water\n\n3. **Equipment:**\n   - Beakers and stirring rods\n   - Heat source (furnace or hotplate)\n   - Fume hood (for handling TiCl4 due to its corrosive nature)\n   - Calcination furnace\n   - Weighing scale\n\n### Synthesis Steps:\n\n#### Step 1: Preparation of Solutions\n1. **Preparing La(NO3)3·6H2O Solution:**\n   - Weigh an appropriate amount of La(NO3)3·6H2O to prepare a 0.1 M solution. For example, to make 1 liter of a 0.1 M solution:\

In [9]:
from datasets import Dataset

tokenized_dataset = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
# Convert tokenized dataset to Hugging Face Dataset
dataset = Dataset.from_dict(tokenized_dataset)

# Split the dataset into training and test sets
train_dataset, test_dataset = dataset.train_test_split(test_size=0.2, seed=42).values()

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./Llama-3.1-8B-Instruct",
    learning_rate=1e-3,
    num_train_epochs=1,
    weight_decay=0.01,
)



In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()


  0%|          | 0/10 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/junchen/anaconda3/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

: 

In [11]:
trainer.train()

  0%|          | 0/12 [00:00<?, ?it/s]

RuntimeError: stack expects each tensor to be equal size, but got [1, 11, 4, 3, 560, 560] at entry 0 and [1, 29, 4, 3, 560, 560] at entry 1